## AIHub Json Parsing

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [ ]:
%pip install ray

In [1]:
import re
import os
import kss
import ray
import json
from mecab import MeCab
import pandas as pd
from glob import glob
from itertools import chain

In [2]:
pwd

'c:\\Users\\MinSeok\\Documents\\AIHUB'

### Basic Function

In [3]:
def sorted_list(path_list):
    path_list = sorted(path_list, reverse=False)
    path_list = sorted(path_list, key=len)
    
    return path_list

In [6]:
def divide_source_file_list(l, n): 
    
  for i in range(0, len(l), n): 
    yield l[i:i + n] 

In [4]:
def json_file_path_list(path_list):
    
    file_path  = [glob(i, recursive = True) for i in path_list][0]
    file_path = sorted_list(file_path)
    
    return file_path

In [5]:
def train_valid_json_file_path_list(path_list):

  train_file_path, valid_file_path = [glob(i, recursive = True) if 'rain' in i
                                      else glob(i, recursive = True)
                                      for i in path_list]

  train_file_path = sorted_list(train_file_path)
  valid_file_path = sorted_list(valid_file_path)
    
  return train_file_path, valid_file_path

In [7]:
def txt_file_path_list(source_file_nested_list, folder_corpus_type_path):

  text_file_path_list = [folder_corpus_type_path + str(i) + ".txt"
                              for i in range(len(source_file_nested_list))]
    
  return text_file_path_list

In [8]:
def post_txt_file_name_list(corpus_list):
   
  post_corpus_list = [corpus_file.replace("pro", "post")
                      for corpus_file in corpus_list]

  return post_corpus_list

In [ ]:
def make_train_valid_json_txt_file_path_list(json_path_list, txt_path_list)

    train_file_path, valid_file_path = train_valid_json_file_path_list(json_path_list)
    
    the_number_of_json_file = len(train_file_path) + len(valid_file_path)
    print("The number of file:", the_number_of_json_file)
    
    train_text_file_path_list = txt_file_path_list(train_file_path, txt_path_list[0])
    valid_text_file_path_list = txt_file_path_list(valid_file_path, txt_path_list[1])

    return train_file_list, valid_file_list, train_text_file_path_list, valid_text_file_path_list

In [ ]:
def make_json_txt_file_path_list(json_path_list, txt_path_list)

    file_list = json_file_path_list(json_path_list)
    
    the_number_of_json_file = len(file_path) 
    print("The number of file:", the_number_of_json_file)
    
    text_file_path_list = txt_file_path_list(train_file_path txt_path_list)

    return file_list, text_file_path_list

In [61]:
def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거


    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = source.replace(source[:2], "")
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거 
    
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False
    

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외


            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거


            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                preprocessing_sentence_list.append(sentence)

    return preprocessing_sentence_list

### AIHUB 문서요약 텍스트

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=97)

#### Convert JSON File to TXT File

In [ ]:
json_path_list = ['AIHUB_문서요약 텍스트/Training/'+ '/**/*.json', 
                  'AIHUB_문서요약 텍스트/Validation/'+ '/**/*.json']
txt_path_list = ["exploration/document_summary_text_pro/AIHUB_document_summary_text_train_", 
                 "exploration/document_summary_text_pro/AIHUB_document_summary_text_valid_"]

In [ ]:
train_file_list, valid_file_list, train_text_file_path_list, valid_text_file_path_list = \
    make_train_valid_json_txt_file_path_list(json_path_list, txt_path_list)

In [ ]:
def make_source_list(json_sample):
    
    source_df = pd.DataFrame(json_sample['documents'])
    source_dict = dict(source_df['text'].explode())
    source_json = pd.json_normalize(source_dict)  

    source_list = []
    for source_index in source_json:
        for source_nested_list in source_json[source_index]:
            for source_single_list in source_nested_list:
                source_sentence = ""
                for source_single in source_single_list:
                    if type(source_single) == dict:
                        for key, value in source_single.items():
                            if key == "sentence":
                                source_sentence += " " + value
                if len(source_sentence) > 0:
                    source_list.append(source_sentence)
                        
    return source_list

In [13]:
def count_number_of_txt_file_with_batch_list(source_file_list, batch_size):
    
    the_number_of_total_txt_file = 0
    the_number_of_txt_file_list = []
    
    for i in range(len(source_file_list)):    
        
        source_file = source_file_list[i] 
            

        with open(source_file, 'r', encoding='utf-8') as one_json_file:
            one_json_sample = json.load(one_json_file)
            
        source_list = make_source_list(one_json_sample)

        the_number_of_txt_file = ((len(source_list) // batch_size) + 1)
        
        if len(source_list) >= batch_size:
            the_number_of_txt_file = ((len(source_list) // batch_size) + 1)
            print("File:", source_file)    
            print("Index:", i, "  ", "Length of Source List:", len(source_list), \
            "  ", "The number of txt file:", the_number_of_txt_file, "\n")
            the_number_of_txt_file_list.append(the_number_of_txt_file)
            the_number_of_total_txt_file +=  the_number_of_txt_file
        else:
            the_number_of_total_txt_file += 1
            the_number_of_txt_file_list.append(1)
            print("[For Example]")
            print("This is not subject of batch. It's small source list")                            
            print("File:", source_file)
            print("Length of Source List:", len(source_list), "\n")

    print("Batch Size:", batch_size)
    print("The number of txt file:", the_number_of_total_txt_file)
    
    return the_number_of_total_txt_file, the_number_of_txt_file_list

In [14]:
def write_jsontext_to_txt_file_with_batch_list(source_file_list,
                                    text_file_path_list,
                                    batch_size, the_number_of_txt_file_list):

  print("[Size]")
  print("The number of preprocessing corpus: " + str(sum(the_number_of_txt_file_list)))
  print("\n[Order]")
  num = 0
  
    for i in range(len(source_file_list)):

      source_file = source_file_list[i]
      
      with open(source_file, 'r', encoding='utf-8') as one_json_file:
        one_json_sample = json.load(one_json_file)

      source_list = make_source_list(one_json_sample)
      
      n = batch_size
      source_batch_list = list(divide_source_file_list(source_list, n))
      
      for source_list in source_batch_list:
          num += 1
          print(str(num), end=" ")  
          
          with open(os.path.join('AIHUB_corpus/' + text_file_path_list[i][:-4] + "_" + str(num) + ".txt"), "a", encoding='utf-8') as fp:        
              fp.write("\n".join(source_list))   
                

In [15]:
batch_size = 1000
the_number_of_train_txt_file, the_number_of_train_txt_file_list = count_number_of_txt_file_with_batch_list(train_file_list, batch_size)

File: AIHUB_문서요약 텍스트/Training\법률_train_original\train_original.json
Index: 0    Length of Source List: 42178    The number of txt file: 43 

File: AIHUB_문서요약 텍스트/Training\사설_train_original\train_original.json
Index: 0    Length of Source List: 242550    The number of txt file: 243 

File: AIHUB_문서요약 텍스트/Training\신문기사_train_original\train_original.json
Index: 0    Length of Source List: 2187380    The number of txt file: 2188 

Batch Size: 1000
The number of txt file: 2474


In [16]:
batch_size = 1000
the_number_of_valid_txt_file, the_number_of_valid_txt_file_list = count_number_of_txt_file_with_batch_list(valid_file_list, batch_size)

File: AIHUB_문서요약 텍스트/Validation\법률_valid_original\valid_original.json
Index: 0    Length of Source List: 4780    The number of txt file: 5 

File: AIHUB_문서요약 텍스트/Validation\사설_valid_original\valid_original.json
Index: 0    Length of Source List: 25202    The number of txt file: 26 

File: AIHUB_문서요약 텍스트/Validation\신문기사_valid_original\valid_original.json
Index: 0    Length of Source List: 239892    The number of txt file: 240 

Batch Size: 1000
The number of txt file: 271


In [17]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(train_file_list, train_text_file_path_list,
                batch_size, the_number_of_train_txt_file_list)

[Size]
The number of preprocessing corpus: 2474

[Order]
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263

In [18]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(valid_file_list, valid_text_file_path_list,
                batch_size, the_number_of_valid_txt_file_list)

[Size]
The number of preprocessing corpus: 271

[Order]
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 

#### Preprocess TXT File

In [ ]:
def post_txt_file_path_list(corpus_path_list):
   
  post_corpus_path_list = [corpus_file.replace("pro", "post")
                      for corpus_file in corpus_path_list]

  return post_corpus_path_list

In [ ]:
def make_pro_post_txt_file_path_list(pro_corpus_path):
    
    pro_total_corpus_path_list = glob(pro_corpus_path)
    pro_total_corpus_path_list = sorted_list(pro_total_corpus_path_list)
    post_total_corpus_path_list = post_txt_file_path_list(pro_total_corpus_path_list)

    return pro_total_corpus_path_list, post_total_corpus_path_list

In [ ]:
pro_corpus_path = "AIHUB_corpus/exploration/document_summary_text_pro/AIHUB_document_summary_text_" + "*.txt"
pro_total_corpus_path_list, post_total_corpus_path_list = make_pro_post_txt_file_name_list(pro_corpus_path)

In [20]:
len(pro_total_corpus_path_list)

2745

In [63]:
line_list = []
line_num = 0
with open(pro_total_corpus_path_list[0], 'r', encoding='utf-8') as f:
    lines = f.read().splitlines() 
    for line in lines:
        line_num += 1
        if line_num <= 1:
           line_list.append(line)
for line in line_list:
    print(line, end="\n\n")

 가. 부가가치세법 제22조 제3항 단서에 제1호와 제2호가 동시에 해당한다는 뜻은 제18조의 예정신고와 예정납부끼리, 제19조의 확정신고와 확정납부끼리 동시에 해당하는 경우를 말하는 것이지 제18조의 예정신고나 그 납부와 제19조의 확정신고나 그 납부가 동시에 해당하는 경우를 가리키는 것이 아니라고 해석되므로 부가가치세의 예정신고 또는 그 납부를 아니하고 또한 확정신고 또는 그 납부를 아니한 경우 통털어서 한번만 가산세를 부과하는 것이 아니라 각각 독립하여 가산세부과대상이 된다.



In [64]:
line_list = []
line_num = 0
with open(pro_total_corpus_path_list[0], 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()
    for line in lines:
        line_num += 1
        if line_num <= 1:  
            sentences = formal_preprocessing_text(line)
            for sentence in sentences:
                line_list.append(sentence) 
            
for line in line_list:
    print(line, end="\n\n")

부가가치세법 제22조 제3항 단서에 제1호와 제2호가 동시에 해당한다는 뜻은 제18조의 예정신고와 예정납부끼리, 제19조의 확정신고와 확정납부끼리 동시에 해당하는 경우를 말하는 것이지 제18조의 예정신고나 그 납부와 제19조의 확정신고나 그 납부가 동시에 해당하는 경우를 가리키는 것이 아니라고 해석되므로 부가가치세의 예정신고 또는 그 납부를 아니하고 또한 확정신고 또는 그 납부를 아니한 경우 통털어서 한번만 가산세를 부과하는 것이 아니라 각각 독립하여 가산세부과대상이 된다.



In [65]:
ray.init(num_cpus = 4)

@ray.remote
def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거


    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = source.replace(source[:2], "")
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거 
    
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False
    

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외


            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거


            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                preprocessing_sentence_list.append(sentence)

    return preprocessing_sentence_list

2023-05-31 17:08:20,288	INFO worker.py:1625 -- Started a local Ray instance.


In [ ]:
def preprocessing_corpus_txt(pro_total_corpus_path_list, post_total_corpus_path_list):
    print("[Size]")
    print("The number of preprocessing corpus: " + str(len(pro_total_corpus_path_list)))
    print("\n[Order]")
    num = 0
    process_num = 10    

    for pro, post in zip(pro_total_corpus_path_list, post_total_corpus_path_list):

        sentence_list = []

        with open(pro, 'r', encoding='utf-8') as f:
            lines = f.read().splitlines() 
            nested_lines_num = len(lines) // process_num
            for i in range(nested_lines_num - 1):
                start_line = process_num * i
                end_line = process_num * (i+1)
                futures = [formal_preprocessing_text.remote(lines[start_line:end_line][j]) for j in range(process_num)]
                results = ray.get(futures)

                if i == nested_lines_num - 2:
                    futures = [formal_preprocessing_text.remote(lines[end_line:][j]) for j in range(len(lines) - end_line)]
                    results = ray.get(futures)

                sentences = list(chain.from_iterable(results))
                sentence_list.append(sentences)

        sentence_list = list(chain.from_iterable(sentence_list))

        num += 1
        print(str(num), end=" ")  

        with open(post, 'a', encoding='utf-8') as fp:
            fp.write("\n".join(sentence_list))

In [66]:
preprocessing_corpus_txt(pro_total_corpus_path_list, post_total_corpus_path_list)

[Size]
The number of preprocessing corpus: 2745

[Order]
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263

In [67]:
ray.shutdown()

In [ ]:
def merge_and_deduplicate_corpus_txt(preprocessing_corpus_path, merge_corpus_path,
                                      deduplicate_corpus_path):
    
    corpus_list = glob(preprocessing_corpus_path)
    corpus_list = sorted_list(corpus_list)
    
    with open(merge_corpus_path, 'w', encoding='utf-8') as f:
        for corpus in corpus_list:
            with open(corpus, encoding='utf-8') as text:
                for line in text:
                    f.write(line)
                    
    with open(deduplicate_corpus_path, 'w', encoding='utf-8') as f1:
        with open(merge_corpus_path, encoding='utf-8') as f2:
            lines = f2.read().splitlines()
            single_sentence_dict = dict.fromkeys(lines)
            single_sentence_list = list(single_sentence_dict)
            f1.write("\n".join(single_sentence_list))                

In [ ]:
preprocessing_corpus_path = "AIHUB_corpus/exploration/document_summary_text_post/AIHUB_document_summary_text_" +"*.txt"
merge_corpus_path = 'AIHUB_corpus/duplicate/AIHUB_document_summary_text.txt'
deduplicate_corpus_path = 'AIHUB_corpus/AIHUB_document_summary_text.txt'

In [ ]:
merge_and_deduplicate_corpus_txt(preprocessing_corpus_path, merge_corpus_path, 
                                  deduplicate_corpus_path)